In [ ]:
%%capture
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

# Initialize Model

In [ ]:
#initialize model and tokenizer

from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None # None for auto detection
load_in_4bit = False # Use 4bit quantization to reduce memory usage


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    token = "hf_...", # enter token here
)

#load adapters
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.11.10: Fast Llama patching. Transformers:4.46.2.
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 8.9. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

Unsloth 2024.11.10 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


# Dataset Functions
- no_scratchpad(x,y)
- simplified_scratchpad(x,y)
- detailed_scratchpad(x,y)
- generate_dataset_dict(num_examples, max_length, scratchpad, skew)

In [ ]:
#Dataset Function

def no_scratchpad(x,y):
    output = []
    output.append({"role": "user", "content": f"{x}+{y}="})
    output.append({"role": "assistant", "content": f"{x+y}"})
    return output

def simplified_scratchpad(x, y):
    # Convert numbers to strings and pad with zeros to match lengths
    x_str = str(x)
    y_str = str(y)
    max_len = max(len(x_str), len(y_str))
    x_str = x_str.zfill(max_len)
    y_str = y_str.zfill(max_len)

    # Initialize variables
    carry = 0
    result = []
    steps = []

    # Loop from least significant digit to most significant digit
    for i in range(max_len - 1, -1, -1):
        digit_x = int(x_str[i])
        digit_y = int(y_str[i])
        total = digit_x + digit_y + carry
        digit_sum = total % 10
        carry = total // 10

        # Insert at the beginning since we're building the result from LSB to MSB
        result.insert(0, str(digit_sum))
        steps.append(f"A->{digit_sum} , C->{carry}")

    # Handle remaining carry
    if carry > 0:
        result.insert(0, str(carry))
        steps.append(f"A->{carry} , C->0")

    # Reverse the steps to display from least significant digit first
    scratchpad_str = "\n".join(steps)

    # Build the final result string
    result_str = "".join(result)

    # Prepare the output
    output = []
    output.append({"role": "user", "content": f"{x}+{y}="})
    output.append({"role": "assistant", "content": f"{scratchpad_str}\n{result_str}"})
    return output

def detailed_scratchpad(x, y):
    # Convert numbers to strings and store original strings
    x_str_orig = str(x)
    y_str_orig = str(y)

    # Pad with zeros to match lengths
    max_len = max(len(x_str_orig), len(y_str_orig))
    x_str = x_str_orig.zfill(max_len)
    y_str = y_str_orig.zfill(max_len)

    # Initialize variables
    carry = 0
    result = []
    steps = ["<scratch>"]

    # Add the operands and their digit counts using original strings
    steps.append(f"[{', '.join(x_str_orig)}] has {len(x_str_orig)} digits")
    steps.append(f"[{', '.join(y_str_orig)}] has {len(y_str_orig)} digits")

    # Loop from least significant digit to most significant digit
    for i in range(max_len -1, -1, -1):
        # Current digits
        digit_x = int(x_str[i])
        digit_y = int(y_str[i])

        # Build the intermediate strings with commas between digits
        left_x_commas = ', '.join(x_str[:i+1])
        left_y_commas = ', '.join(y_str[:i+1])

        # Copy current result before appending new digit
        A_current = result.copy()

        # For the first step, do not show 'A='
        if i == max_len - 1:
            step = f"[{left_x_commas}] + [{left_y_commas}] , C={carry}; {digit_x}+{digit_y}+{carry}={digit_x+digit_y+carry} , A->{(digit_x+digit_y+carry)%10} , C->{(digit_x+digit_y+carry)//10}"
        else:
            step = f"[{left_x_commas}] + [{left_y_commas}] , A= {A_current} , C={carry}; {digit_x}+{digit_y}+{carry}={digit_x+digit_y+carry} , A->{(digit_x+digit_y+carry)%10} , C->{(digit_x+digit_y+carry)//10}"

        # Append the step before updating result and carry
        steps.append(step)

        # Perform the calculation
        total = digit_x + digit_y + carry
        digit_sum = total % 10
        new_carry = total // 10

        # Update 'A' after calculation
        result.insert(0, digit_sum)

        # Update carry
        carry = new_carry

    # After loop, handle remaining carry
    if carry > 0:
        # Copy current result before appending carry
        A_current = result.copy()
        # Append the step
        step = f"[] + [] , A= {A_current} , C={carry}; 0+0+{carry}={carry} , A->{carry} , C->0"
        steps.append(step)
        # Append the carry to the result
        result.insert(0, carry)
        carry = 0

    # End of scratchpad
    steps.append(f"[] + [] , A= {result} , C={carry}; END")
    steps.append("</scratch>")

    # Build the final result string
    result_str = ''.join(map(str, result))

    # Add the result to steps
    steps.append(result_str)

    # Prepare the output
    scratchpad_str = "\n".join(steps)
    output = []
    output.append({"role": "user", "content": f"{x}+{y}="})
    output.append({"role": "assistant", "content": scratchpad_str})
    return output

def generate_dataset_dict(num_examples, max_length, scratchpad, skew):
    k = skew
    dataset_dict = {'text':[]}
    for _ in range(num_examples):
        a = max(random.randint(1, max_length) for _ in range(k))
        b = max(random.randint(1, max_length) for _ in range(k))
        x = random.randint(10**(a-1), 10**(a)-1)
        y = random.randint(10**(b-1), 10**(b)-1)
        templated_str = tokenizer.apply_chat_template(scratchpad(x,y), tokenize = False)

        dataset_dict['text'].append(templated_str)
    return dataset_dict

# Finetune function
finetune(max_length, num_examples, scratchpad, skew)

In [ ]:
from datasets import Dataset
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported
from unsloth.chat_templates import train_on_responses_only

def finetune(max_length, num_examples, scratchpad, skew):
    #make dataset
    max_length = max_length #max length of operand
    num_examples = num_examples #length of dataset
    scratchpad = scratchpad #"simplified" or "detailed" or "no"
    skew = skew #the higher the number, the more the dataset is skewed towards higher digits

    dataset = Dataset.from_dict(generate_dataset_dict(num_examples, max_length, scratchpad, skew))

    #initialize trainer
    trainer = SFTTrainer(
        model = model,
        tokenizer = tokenizer,
        train_dataset = dataset,
        dataset_text_field = "text", #HERE you specify the key which holds the text
        max_seq_length = max_seq_length,
        data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
        dataset_num_proc = 2,
        packing = False, # Can make training 5x faster for short sequences.
        args = TrainingArguments(
            per_device_train_batch_size = 32,
            gradient_accumulation_steps = 2,
            warmup_steps = 5,
            num_train_epochs = 1, # Set this for 1 full training run.
            #max_steps = ,
            learning_rate = 2e-4,
            fp16 = not is_bfloat16_supported(),
            bf16 = is_bfloat16_supported(),
            logging_steps = 1,
            optim = "adamw_8bit",
            weight_decay = 0.01,
            lr_scheduler_type = "linear",
            seed = 3407,
            output_dir = "outputs",
            save_strategy = "steps",
            save_steps = 125,
            report_to = "none", # Use this for WandB etc

        ),
    )

    trainer = train_on_responses_only(
        trainer,
        instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
        response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
    )

    trainer_stats = trainer.train()

# MAIN

In [ ]:
max_length = 10
num_examples = 300000
scratchpad = simplified_scratchpad #choose between no_scratchpad, simplified_scratchpad, detailed_scratchpad
skew = 2 #decides amount of bias towards large numbers of the sampling process. 2 is best for scratchpads, 1 is best for no scratchpads

finetune(max_length, num_examples, scratchpad, skew)

wandb: Currently logged in as: leon-li2000 (studypanda). Use `wandb login --relogin` to force relogin


Map (num_proc=2):   0%|          | 0/300000 [00:00<?, ? examples/s]

Map:   0%|          | 0/300000 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 300,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 32 | Gradient Accumulation steps = 2
\        /    Total batch size = 64 | Total steps = 4,687
 "-____-"     Number of trainable parameters = 24,313,856


Step,Training Loss
1,1.825700
2,1.870900
3,1.825700
4,1.781600
5,1.574300
6,1.315000
7,1.092800
8,0.892700
9,0.716100
10,0.622300


{'a': 1, 'b': 1, 'accuracy': 1.0}
{'a': 1, 'b': 2, 'accuracy': 1.0}
{'a': 1, 'b': 3, 'accuracy': 1.0}
{'a': 1, 'b': 4, 'accuracy': 1.0}
{'a': 1, 'b': 5, 'accuracy': 1.0}
{'a': 1, 'b': 6, 'accuracy': 1.0}
{'a': 1, 'b': 7, 'accuracy': 1.0}
{'a': 1, 'b': 8, 'accuracy': 1.0}
{'a': 1, 'b': 9, 'accuracy': 1.0}
{'a': 1, 'b': 10, 'accuracy': 1.0}
{'a': 2, 'b': 1, 'accuracy': 1.0}
{'a': 2, 'b': 2, 'accuracy': 1.0}
{'a': 2, 'b': 3, 'accuracy': 1.0}
{'a': 2, 'b': 4, 'accuracy': 1.0}
{'a': 2, 'b': 5, 'accuracy': 1.0}
{'a': 2, 'b': 6, 'accuracy': 1.0}
{'a': 2, 'b': 7, 'accuracy': 1.0}
{'a': 2, 'b': 8, 'accuracy': 1.0}
{'a': 2, 'b': 9, 'accuracy': 1.0}
{'a': 2, 'b': 10, 'accuracy': 1.0}
{'a': 3, 'b': 1, 'accuracy': 1.0}
{'a': 3, 'b': 2, 'accuracy': 1.0}
{'a': 3, 'b': 3, 'accuracy': 1.0}
{'a': 3, 'b': 4, 'accuracy': 1.0}
{'a': 3, 'b': 5, 'accuracy': 1.0}
{'a': 3, 'b': 6, 'accuracy': 1.0}
{'a': 3, 'b': 7, 'accuracy': 1.0}
{'a': 3, 'b': 8, 'accuracy': 1.0}
{'a': 3, 'b': 9, 'accuracy': 1.0}
{'a': 3, 'b'

accuracy,▁
accuracy,0.9996
